In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 62.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pulp import *
import itertools

# Data Analysis

In [ ]:
staff_df = pd.read_csv("/content/data.csv")

In [ ]:
def analyze_staff_data(staff_df):
    print("Staff Data Summary:")
    print(staff_df['Role'].value_counts())
    print("\nDepartment Distribution:")
    print(staff_df['Department'].value_counts())
    print("\nSeniority Statistics:")
    print(staff_df['Seniority'].describe())

    for role in ['Main Doctor', 'Assistant Doctor', 'Nurse']:
        print(f"\n{role} Department Distribution:")
        print(staff_df[staff_df['Role'] == role]['Department'].value_counts())

analyze_staff_data(staff_df)

Staff Data Summary:
Role
Nurse               44
Main Doctor         19
Assistant Doctor    17
Name: count, dtype: int64

Department Distribution:
Department
Emergency            23
Clinical             22
Surgery              21
Internal Medicine    14
Name: count, dtype: int64

Seniority Statistics:
count    80.000000
mean     10.612500
std       5.990486
min       1.000000
25%       5.750000
50%      10.500000
75%      15.250000
max      22.000000
Name: Seniority, dtype: float64

Main Doctor Department Distribution:
Department
Surgery              6
Emergency            6
Clinical             4
Internal Medicine    3
Name: count, dtype: int64

Assistant Doctor Department Distribution:
Department
Clinical             7
Surgery              4
Emergency            4
Internal Medicine    2
Name: count, dtype: int64

Nurse Department Distribution:
Department
Emergency            13
Surgery              11
Clinical             11
Internal Medicine     9
Name: count, dtype: int64


In [ ]:
def check_staff_sufficiency(staff_df, num_days=61):
    total_shifts = num_days * 2  # 2 shifts per day
    required_staff_shifts = {
        'Main Doctor': total_shifts,
        'Assistant Doctor': total_shifts,
        'Nurse': total_shifts * 2  # 2 nurses per shift
    }

    available_staff = staff_df['Role'].value_counts()

    print("Required staff per shift:")
    for role, count in required_staff_shifts.items():
        print(f"{role}: {count/total_shifts}")

    print("\nAvailable staff:")
    print(available_staff)

    for role in required_staff_shifts:
        if role not in available_staff or available_staff[role] < required_staff_shifts[role]/total_shifts:
            print(f"\nInsufficient {role}s. Need at least {required_staff_shifts[role]/total_shifts:.2f}, have {available_staff.get(role, 0)}")
            return False

    print("\nSufficient staff available for all roles.")
    return True

check_staff_sufficiency(staff_df)

Required staff per shift:
Main Doctor: 1.0
Assistant Doctor: 1.0
Nurse: 2.0

Available staff:
Role
Nurse               44
Main Doctor         19
Assistant Doctor    17
Name: count, dtype: int64

Sufficient staff available for all roles.


True

In [ ]:
def analyze_role_department_distribution(staff_df):
    roles = ['Main Doctor', 'Assistant Doctor', 'Nurse']
    departments = staff_df['Department'].unique()

    print("Role-Department Distribution:")
    for role in roles:
        print(f"\n{role}:")
        role_dept_dist = staff_df[staff_df['Role'] == role]['Department'].value_counts()
        for dept in departments:
            print(f"  {dept}: {role_dept_dist.get(dept, 0)}")

analyze_role_department_distribution(staff_df)

Role-Department Distribution:

Main Doctor:
  Internal Medicine: 3
  Surgery: 6
  Emergency: 6
  Clinical: 4

Assistant Doctor:
  Internal Medicine: 2
  Surgery: 4
  Emergency: 4
  Clinical: 7

Nurse:
  Internal Medicine: 9
  Surgery: 11
  Emergency: 13
  Clinical: 11


In [ ]:
def check_department_diversity_feasibility(staff_df):
    roles = ['Main Doctor', 'Assistant Doctor', 'Nurse']
    departments = staff_df['Department'].unique()

    for role in roles:
        role_depts = staff_df[staff_df['Role'] == role]['Department'].unique()
        if len(role_depts) < 3:
            print(f"Warning: {role}s are only available from {len(role_depts)} departments.")
            print(f"Departments with {role}s: {', '.join(role_depts)}")

    all_role_depts = [set(staff_df[staff_df['Role'] == role]['Department']) for role in roles]
    common_depts = set.intersection(*all_role_depts)

    if len(common_depts) < 3:
        print(f"\nWarning: Only {len(common_depts)} departments have staff for all roles.")
        print(f"Departments with all roles: {', '.join(common_depts)}")
        print("This makes it impossible to always have 3 departments represented in each shift.")
    else:
        print("\nIt is possible to have staff from at least 3 departments in each shift.")

check_department_diversity_feasibility(staff_df)


It is possible to have staff from at least 3 departments in each shift.


# Create Model

In [ ]:
def load_staff_df(file_path='/content/data - data.csv'):
    staff_df = pd.read_csv(file_path)
    required_columns = ['ID', 'Name', 'Department', 'Role', 'Seniority']
    if not all(col in staff_df.columns for col in required_columns):
        raise ValueError(f"The CSV file must contain the following columns: {required_columns}")
    return staff_df

In [ ]:
def create_model(staff_df, num_days=61):
    # Sets
    D = range(1, num_days + 1)  # Days
    S = ['day', 'night']  # Shifts
    R = ['Main Doctor', 'Assistant Doctor', 'Nurse']  # Roles
    E = staff_df['ID'].tolist()  # Employees
    P = staff_df['Department'].unique().tolist()  # Departments

    # Parameters
    role_requirements = {'Main Doctor': 1, 'Assistant Doctor': 1, 'Nurse': 2}
    employee_roles = {e: staff_df.loc[staff_df['ID'] == e, 'Role'].values[0] for e in E}
    employee_departments = {e: staff_df.loc[staff_df['ID'] == e, 'Department'].values[0] for e in E}
    employee_seniority = {e: staff_df.loc[staff_df['ID'] == e, 'Seniority'].values[0] for e in E}

    # Create the model
    model = LpProblem("Hospital_Scheduling", LpMinimize)

    # Decision Variables
    x = LpVariable.dicts("shift", ((e, d, s) for e in E for d in D for s in S), cat='Binary')

    # Variables for shift count per employee
    shift_count = LpVariable.dicts("shift_count", E, lowBound=0, cat='Integer')

    # Variables for maximum shifts
    max_shifts = LpVariable("max_shifts", lowBound=4, cat='Integer')

    # min_shifts = 4
    z = max_shifts

    # Objective Function: Minimize the difference between max and min shifts
    model += z, "Minimize_max_shifts"

    # Constraints
    # 1. Each shift must have the required number of staff for each role (HARD CONSTRAINT)
    for d in D:
        for s in S:
            for r in R:
                model += lpSum(x[e, d, s] for e in E if employee_roles[e] == r) == role_requirements[r]

    # 2. Each employee can work at most one shift per day (ADDITIONAL HARD CONSTRAINT)
    for e in E:
        for d in D:
            model += lpSum(x[e, d, s] for s in S) <= 1

    # 3. No day shift immediately after night shift (ADDITIONAL HARD CONSTRAINT)
    for e in E:
        for d in range(1, num_days):
            model += x[e, d, 'night'] + x[e, d+1, 'day'] <= 1

    # 4. No 3 consecutive working days (HARD CONSTRAINT)
    for e in E:
        for d in range(1, num_days - 1):  # We need to look at 3 consecutive days
            model += lpSum(x[e, d, s] for s in S) + lpSum(x[e, d+1, s] for s in S) + lpSum(x[e, d+2, s] for s in S) <= 2

    # 5. Each employee has at least 4 shifts (ADDITIONAL HARD CONSTRAINT)
    for e in E:
        model += shift_count[e] >= 4

    # 6. The maximum number of shifts for any employee should be minimized (ADDITIONAL SOFT CONSTRAINT)
    for e in E:
        model += shift_count[e] == lpSum(x[e, d, s] for d in D for s in S)
        model += shift_count[e] <= max_shifts

    # 7. Each shift should have staff from at least 3 departments (SOFT CONSTRAINT)
    dept_diversity = LpVariable.dicts("dept_diversity", ((d, s, p) for d in D for s in S for p in P), cat='Binary')
    for d in D:
        for s in S:
            for p in P:
                model += lpSum(x[e, d, s] for e in E if employee_departments[e] == p) >= dept_diversity[d, s, p]
                model += lpSum(x[e, d, s] for e in E if employee_departments[e] == p) <= len(E) * dept_diversity[d, s, p]

            model += lpSum(dept_diversity[d, s, p] for p in P) >= 3

    # 8. At least 90% of shifts should have at least one staff with high seniority (SOFT CONSTRAINT)
    high_seniority_shifts = LpVariable.dicts("high_seniority", ((d, s) for d in D for s in S), cat='Binary')
    for d in D:
        for s in S:
            model += lpSum(x[e, d, s] for e in E if employee_seniority[e] > 10) >= high_seniority_shifts[d, s]
    model += lpSum(high_seniority_shifts[d, s] for d in D for s in S) >= 0.9 * len(D) * len(S)

    return model, x


In [ ]:
def evaluate_schedule(schedule_df, staff_df, expected_shifts=122):
    # Total number of shifts
    total_shifts = len(schedule_df)

    # Shifts per employee
    shifts_per_employee = schedule_df['Employee_ID'].value_counts()
    max_shifts = shifts_per_employee.max()
    min_shifts = shifts_per_employee.min()
    avg_shifts = shifts_per_employee.mean()
    shift_variance = shifts_per_employee.var()

    # Role coverage (PRIORITY)
    role_coverage = schedule_df.groupby(['Day', 'Shift', 'Role']).size().unstack(fill_value=0)
    correct_role_coverage = (
        (role_coverage['Main Doctor'] == 1) &
        (role_coverage['Assistant Doctor'] == 1) &
        (role_coverage['Nurse'] == 2)
    ).all()

    # Department coverage
    def count_departments(group):
        return group['Department'].nunique()
    dept_diversity = schedule_df.groupby(['Day', 'Shift']).apply(count_departments)
    avg_dept_diversity = dept_diversity.mean()

    # Senior coverage
    def has_senior_staff(group):
        return (group['Employee_ID'].map(staff_df.set_index('ID')['Seniority']) > 10).any()
    senior_coverage = schedule_df.groupby(['Day', 'Shift']).apply(has_senior_staff)
    senior_coverage_pct = senior_coverage.mean() * 100

    # Night-Day shifts
    night_day_shifts = schedule_df.sort_values(['Employee_ID', 'Day', 'Shift'])
    night_day_shifts['next_day'] = night_day_shifts.groupby('Employee_ID')['Day'].shift(-1)
    night_day_shifts['next_shift'] = night_day_shifts.groupby('Employee_ID')['Shift'].shift(-1)
    night_day_count = (
        (night_day_shifts['Day'] == night_day_shifts['next_day'] - 1) &
        (night_day_shifts['Shift'] == 'night') &
        (night_day_shifts['next_shift'] == 'day')
    ).sum()

    # Consecutive working days
    def count_consecutive_working_days(group):
        working_days = group['Day'].sort_values().unique()
        if len(working_days) == 0:
            return 0
        day_diff = np.diff(working_days)
        consecutive = (day_diff == 1)
        runs = [len(list(g)) for k, g in itertools.groupby(consecutive) if k]
        return max(runs) + 1 if runs else 1

    max_consecutive_days = schedule_df.groupby('Employee_ID').apply(count_consecutive_working_days).max()

    # Fairness metric
    fairness_metric = 1 - (shift_variance / (avg_shifts ** 2))

    # Evaluation
    evaluation = {
        "Correct Role Coverage (PRIORITY)": correct_role_coverage,
        "Total Shifts": total_shifts / 4,
        "Expected Shifts": expected_shifts,
        "Shift Coverage (%)": ((total_shifts / 4) / expected_shifts) * 100,
        "Max Shifts per Employee": max_shifts,
        "Min Shifts per Employee": min_shifts,
        "Average Shifts per Employee": avg_shifts,
        "Shift Variance": shift_variance,
        "Fairness Metric": fairness_metric,
        "Average Department Diversity per Shift": avg_dept_diversity,
        "Senior Staff Coverage (%)": senior_coverage_pct,
        "Number of Consecutive Night-Day Shifts": night_day_count,
        "Max Consecutive Working Days of any Employee": max_consecutive_days
    }

    return evaluation


In [ ]:
def solve_and_extract_schedule(model, x, staff_df):
    model.solve(PULP_CBC_CMD(msg=0))

    schedule = []
    for (e, d, s), var in x.items():
        if var.value() == 1:
            employee = staff_df[staff_df['ID'] == e].iloc[0]
            schedule.append({
                'Day': d,
                'Shift': s,
                'Employee_ID': e,
                'Name': employee['Name'],
                'Department': employee['Department'],
                'Role': employee['Role']
            })

    return pd.DataFrame(schedule)

In [ ]:
if __name__ == "__main__":
    try:
        staff_df = pd.read_csv('/content/data.csv')
        model, x = create_model(staff_df)

        solver = PULP_CBC_CMD(msg=0)
        model.solve(solver)

        schedule = []
        for (e, d, s), var in x.items():
            if var.value() == 1:
                employee = staff_df[staff_df['ID'] == e].iloc[0]
                schedule.append({
                    'Day': d,
                    'Shift': s,
                    'Employee_ID': e,
                    'Name': employee['Name'],
                    'Department': employee['Department'],
                    'Role': employee['Role']
                })

        schedule_df = pd.DataFrame(schedule)

        evaluation_results = evaluate_schedule(schedule_df, staff_df)
        print("\nSchedule Evaluation:")
        for metric, value in evaluation_results.items():
            print(f"{metric}: {value}")

        schedule_df.to_csv('optimized_hospital_schedule.csv', index=False)
        print("\nOptimized schedule saved to 'optimized_hospital_schedule.csv'")

    except Exception as e:
        print(f"An error occurred: {e}")


Schedule Evaluation:
Correct Role Coverage (PRIORITY): True
Total Shifts: 122.0
Expected Shifts: 122
Shift Coverage (%): 100.0
Max Shifts per Employee: 8
Min Shifts per Employee: 4
Average Shifts per Employee: 6.1
Shift Variance: 2.4962025316455696
Fairness Metric: 0.932915814790498
Average Department Diversity per Shift: 3.3278688524590163
Senior Staff Coverage (%): 98.36065573770492
Number of Consecutive Night-Day Shifts: 0
Max Consecutive Working Days of any Employee: 2

Optimized schedule saved to 'optimized_hospital_schedule.csv'
